In [114]:
import numpy as np
import copy
from scipy import misc
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, CyclicLR, ReduceLROnPlateau, LinearLR, ExponentialLR
import random
from torch.autograd.functional import jacobian, hessian
# import AUTOGRAD.FUNCTIONAL.JACOBIAN as jacobian
import time
import math

import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
mpl.rcParams['legend.fontsize'] = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Parameters

In [115]:
dim=2
kappa=torch.tensor([1]).to(device)
theta=torch.tensor([0.4]).to(device)
nu=torch.tensor([0.02]).to(device)
lamb=torch.tensor([0.15]).to(device)
eta=torch.tensor([0.5]).to(device)
rho=torch.tensor([0.0]).to(device)

In [116]:
num_sample=2**3   #M
#dim=5
num_time_interval=2    #N
T=torch.tensor([1]).to(device)
delta_t = torch.tensor(T/ num_time_interval)
# t_steps = torch.linspace(0,1,num_time_interval).to(device)
# sqrt_delta_t = torch.sqrt(delta_t).to(device)
sqrt_delta_t = torch.sqrt(delta_t).to(device)
x_init = torch.zeros(dim)
# sigma=torch.sqrt(torch.tensor([2.0]))
# lamb=torch.tensor([1.0]).to(device)
mu=torch.tensor([1.0]).to(device)
# sigma=torch.tensor([1.0]).to(device)
alpha=torch.tensor([1.0]).to(device)
start = torch.tensor([0]).to(device)
end = torch.tensor([1]).to(device)
num_steps=5
num_ite=int(num_time_interval/num_steps)
# kappa=torch.tensor([1]).to(device)
# theta=torch.tensor([0.4]).to(device)
# nu=torch.tensor([0.5]).to(device)
# lamb=torch.tensor([0.6]).to(device)
# eta=torch.tensor([0.5]).to(device)
p=torch.tensor([0.95]).to(device)
# rho=torch.tensor([0]).to(device)
num_runs=1
# x0=torch.ones([1,dim]).to(device)+4
# print(x0)

t_steps = torch.linspace(0,T.item(),num_time_interval+1).to(device)
# print(t_steps)
test_interval=[0.5,1.5]
# v=(torch.ones(num_sample,dim,dim)-0.3).to(device)
# v=(torch.ones(num_sample,dim,dim)).to(device)
v=torch.diag(torch.ones(dim)).to(device)
# print(v)
v=v.unsqueeze(0).repeat(num_sample,1,1)
print(v.shape)
# print(v)


# dt=T/num_time_interval
Dt=torch.zeros((num_sample,1,num_time_interval+1)).to(device)
print(Dt.shape)
Dt[:,:,1:]=delta_t
t=torch.cumsum(Dt,axis=2).to(device)

torch.Size([8, 2, 2])
torch.Size([8, 1, 3])


/var/folders/9f/dlwft6dn7hxg65r7xqyxvk6r0000gn/T/ipykernel_20146/224781277.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  delta_t = torch.tensor(T/ num_time_interval)


# Functions

## Nonlinearity - Full - $h(t,x,p,q,\gamma)$

In [117]:
def H(x,z,gamma): #### out_shape =([M]) |  input:  x_shape=[M,dim,1],  z shape = [M,dim,1], gamma shape= [M,dim,dim]
    result=torch.zeros([x.shape[0]]).to(device)
    for i in range(dim-1):
      result+=kappa[i]*(theta[i]-x[:,i+1,0])*z[:,i+1,0]+0.5*torch.pow(nu[i],2)*gamma[:,i+1,i+1]-rho[i]*lamb[i]*nu[i]*(z[:,0,0]*gamma[:,0,i+1])/gamma[:,0,0] \
      + 0.5*torch.pow(rho[i]*nu[i]*gamma[:,0,i+1],2)/gamma[:,0,0]
      # print(result)
    result1=result-0.5*torch.sum(torch.square(lamb))*torch.pow(z[:,0,0],2)/gamma[:,0,0]
    return result1



## Nonlinarity - semi $(t,x,p,q,\sigma)$

In [118]:


def F(x,z,a): #### out_shape =([M]) |  input:  x_shape=[M,D,1],  z shape = [M,D,1],a_shape= [M,D,D]  #This is for rho=0
  result=torch.zeros([x.shape[0]]).to(device)
  for i in range(dim-1):
    result+=-kappa[i]*(theta[i]-x[:,i+1,0])*z[:,i+1,0]
    # print(result)
  result1=result-torch.sqrt(torch.sum(torch.square(lamb)))*torch.abs(z[:,0,0])*torch.sqrt(a[:,0,0])
  return result1



## Terminal

In [119]:

def g1(x): #out_shape= [M,1,1]   | input:  x_shape=[M,D]
  result=1-torch.exp(-eta*x[:,0])
  # result=torch.reshape(result,(result.shape[0],1,1))
  result=torch.reshape(result,(result.shape[0],1))
  return result



## Dynamic of SDE

In [120]:


def update(data,delta_w,sigma):#output: data=(M,D,1) ,  #input data=(M,D,1), delta_w=[M,D,1], sigma=[M,D,1]
    dx = torch.bmm(sigma,delta_w)
    data = data + dx
    data=data
    return data
  
  


## First derivative

In [121]:

def der1(x,f):  #output= [M,D,1], input: x=[M,D,1]
  Du=torch.zeros(x.shape[0],dim).to(device)
  xin=x.clone().detach()
  xin.requires_grad=True
  # print('xin shape', xin.shape)
  u=f(xin)
  # print(u)
  # print('u/f(xin) shape', u.shape)
  Du=torch.autograd.grad(outputs=[u],inputs=[xin],grad_outputs=torch.ones_like(u),
                          allow_unused=True,retain_graph=True,create_graph=True)[0].unsqueeze(2)
  # print('Du shape before reshaped', Du.shape)
  Du=torch.reshape(Du,(Du.shape[0],dim,1))
  # print('Du shape after reshaped', Du.shape)
  return Du



## First and second derivative

In [122]:


def grad_hessian1(t, x,f_): #output= [M,D,D], #input: x=[M,D], t=[M,1], xt= [M,D+1]
    hess_temp=torch.zeros(x.shape[0],dim,dim).to(device)
    Du=torch.zeros(x.shape[0],dim).to(device)

    xin=x.clone().detach()
    xin.requires_grad=True
    # print("xin_shape", xin.shape)
    tin=t.clone().detach()
    # print("tin shape", tin.shape)
    xt_in=torch.cat((xin,tin),1)
    # print("xt_in shape", xt_in.shape)



    # xt_in.requires_grad=True
    u=f_(xt_in)
    # print("u shape: ",u.shape)
    Du=torch.autograd.grad(outputs=[u],inputs=[xin],grad_outputs=torch.ones_like(u),
                           allow_unused=True,retain_graph=True,create_graph=True)[0].unsqueeze(2)
    # print("Du shape:",Du.shape)
    # print("-----")
    # print(torch.autograd.grad(outputs=[Du[:,0,:]],inputs=[xin],grad_outputs=torch.ones_like(Du[:,0,:]),
    #                        allow_unused=True,retain_graph=True,create_graph=True)[0].shape)
    hess_temp= torch.cat([ torch.autograd.grad(outputs=[Du[:,i,:]],inputs=[xin],grad_outputs=torch.ones_like(Du[:,i,:]),
                           allow_unused=True,retain_graph=True,create_graph=True)[0] for i in range(dim)],1)
    # print("D2u shape:",hess_temp.shape)
    Du = torch.reshape(Du,(Du.shape[0],dim,1))
    # print("Du after reshape:",Du.shape)
    hess_temp=torch.reshape(hess_temp,(hess_temp.shape[0],dim,dim))
    # print("D2u after reshape:",hess_temp.shape)

    # print(hess_temp)
    return Du, hess_temp



##  First derivative

In [123]:


def grad_model(t, x,f_): #output= [M,D,1], #input: x=[M,D], t=[M,1], xt= [M,D+1]
    # hess_temp=torch.zeros(x.shape[0],dim,dim).to(device)
    Du=torch.zeros(x.shape[0],dim).to(device)

    xin=x.clone().detach()
    xin.requires_grad=True
    # print("xin_shape", xin.shape)
    tin=t.clone().detach()
    # print("tin shape", tin.shape)
    xt_in=torch.cat((xin,tin),1)
    # print("xt_in shape", xt_in.shape)

    # xt_in.requires_grad=True
    u=f_(xt_in)
    # print("u shape: ",u.shape)
    Du=torch.autograd.grad(outputs=[u],inputs=[xin],grad_outputs=torch.ones_like(u),
                           allow_unused=True,retain_graph=True,create_graph=True)[0].unsqueeze(2)

    Du = torch.reshape(Du,(Du.shape[0],dim,1))
    # print("Du after reshape:",Du.shape)

    return Du



## Hessian 

In [124]:

def hessian_model(t,x,model):
    hess_temp=torch.zeros(x.shape[0],dim,dim).to(device)
    Du=torch.zeros(x.shape[0],dim).to(device)

    xin=x.clone().detach()
    xin.requires_grad=True
    # print("xin_shape", xin.shape)
    tin=t.clone().detach()
    # print("tin shape", tin.shape)
    xt_in=torch.cat((xin,tin),1)
    # print("xt_in shape", xt_in.shape)
    Du = model(xt_in)
    hess_temp= torch.cat([ torch.autograd.grad(outputs=[Du[:,i]],inputs=[xin],grad_outputs=torch.ones_like(Du[:,i]),
                           allow_unused=True,retain_graph=True,create_graph=True)[0] for i in range(dim)],1)

    # print("D2u shape:",hess_temp.shape)

    hess_temp=torch.reshape(hess_temp,(hess_temp.shape[0],dim,dim))

    # print(hess_temp)
    return hess_temp



## Another Hessian!

In [125]:


def hessian1(x,model2z):  #x=[M,D,1]  output= [M,D,1]
    hess_temp=torch.zeros(x.shape[0],dim,dim).to(device)
    # print("D2u shape:",hess_temp.shape)

    Du=torch.zeros(x.shape[0],dim).to(device)
    xin=x.clone().detach()
    # print('this is xin')
    # print(xin)
    xin.requires_grad=True
    Du = model2z(xin.squeeze(2))
    # print('this is Du')
    # print(Du)
    # print('this is Du shape')
    # print(Du.shape)
    # print('this is Du 1')
    # print(Du[:,1,:])
    hess_temp= torch.cat([ torch.autograd.grad(outputs=[Du[:,i,:]],inputs=[xin],grad_outputs=torch.ones_like(Du[:,i,:]),
                           allow_unused=True,retain_graph=True,create_graph=True)[0] for i in range(dim)],1)

    # print("D2u shape:",hess_temp.shape)

    hess_temp=torch.reshape(hess_temp,(hess_temp.shape[0],dim,dim))

    # print(hess_temp)
    return hess_temp




## Another gradient

In [126]:

def grad_fn2(x,z,a,f,xx=False,r=False,p=False,alpha=False):  #output= [M,D,1], input: x=[M,D,1] z=[M,D,1], a=[M,D,D]
  Du=torch.zeros(x.shape[0],dim).to(device)
  xin=x.clone().detach()
  xin.requires_grad=True
  # print('xin shape', xin.shape)
  zin=z.clone().detach()
  zin.requires_grad=True
  # print('zin shape', zin.shape)
  ain=a.clone().detach()
  ain.requires_grad=True
  # print('ain shape', ain.shape)
  # ain=torch.reshape(ain,(num_sample,dim*dim,1))
  u=f(xin,zin,ain)
  # print(u)
  # print('u/f(xin) shape', u.shape)
  if xx==True:
    Du=torch.autograd.grad(outputs=[u],inputs=[xin],grad_outputs=torch.ones_like(u),
                            allow_unused=True,retain_graph=True,create_graph=True)[0]
    # print('Du shape before reshaped', Du.shape)
    Du=torch.reshape(Du,(Du.shape[0],dim,1))
    # print('Du shape after reshaped', Du.shape)
  if r==True:
    Du=torch.autograd.grad(outputs=[u],inputs=[rin],grad_outputs=torch.ones_like(u),
                            allow_unused=True,retain_graph=True,create_graph=True)[0]
    # print('Du shape before reshaped', Du.shape)
    Du=torch.reshape(Du,(Du.shape[0],1,1))
    # Du=torch.reshape(Du,(Du.shape[0],dim,dim))
    # print('Du shape after reshaped', Du.shape)
  if p==True:
    Du=torch.autograd.grad(outputs=[u],inputs=[zin],grad_outputs=torch.ones_like(u),
                            allow_unused=True,retain_graph=True,create_graph=True)[0]
    # print('Du shape before reshaped', Du.shape)
    Du=torch.reshape(Du,(Du.shape[0],dim,1))
    # print('Du shape after reshaped', Du.shape)
  if alpha==True:
    Du=torch.autograd.grad(outputs=[u],inputs=[ain],grad_outputs=torch.ones_like(u),
                            allow_unused=True,retain_graph=True,create_graph=True)[0]
    # print('Du shape before reshaped', Du.shape)
    Du=torch.reshape(Du,(Du.shape[0],dim,dim))
    # print('Du shape after reshaped', Du.shape)

  return Du




In [127]:



def L_matrix(ite,x,model):  #x=[M,D]
  L=torch.zeros(x.shape[0],dim,dim)

  L[:,0,0]=model[ite](x)
  for i in range(dim-1):
    L[:,i+1,i+1]=nu[i]
  return L


def opt_quad():
  return torch.pow(torch.sum(torch.pow(lamb,2)),0.5)/eta

# def L_matrix_test(ite,x,model1):  #This matrix is a matrix with L00 = 0.6 and L11=nu
#   L=torch.zeros(x.shape[0],dim,dim)
#   L[:,0,0]=opt_quad()
#   for i in range(dim-1):
#     L[:,i+1,i+1]=nu[i]
#   return L

def L_matrix_test(x):  #This matrix is a matrix with L00 = 0.6 and L11=nu
  L=torch.zeros(x.shape[0],dim,dim)
  L[:,0,0]=opt_quad()
  for i in range(dim-1):
    L[:,i+1,i+1]=nu[i]
  return L

def V1(t,x,v):
  # result=(1-torch.exp(-eta*x))*(torch.exp(-0.5*(T-t)*torch.sum(torch.pow(lamb,2))))
  result=-torch.exp(-eta*x-0.5*(T-t)*torch.sum(torch.pow(lamb,2))).to(device)
  return result

def V(t,x,v): #output           |#input:
  result=1-torch.exp(-eta*x-0.5*(T-t)*torch.sum(torch.pow(lamb,2))).to(device)
  return result


def V1(t,x,v):
  # result=(1-torch.exp(-eta*x))*(torch.exp(-0.5*(T-t)*torch.sum(torch.pow(lamb,2))))
  result=-torch.exp(-eta*x-0.5*(T-t)*torch.sum(torch.pow(lamb,2))).to(device)
  return result

def derV(t,x,v): #output           |#input:
  result=eta*torch.exp(-eta*x-0.5*(T-t)*torch.sum(torch.pow(lamb,2))).to(device)
  return result

def hessianV(t,x,v): #output           |#input:
  result=-torch.pow(eta,2)*torch.exp(-eta*x-0.5*(T-t)*torch.sum(torch.pow(lamb,2))).to(device)
  return result

# Solving linear equations

$\begin{cases}-\partial v_t - \dfrac12 (\sigma^\intercal\sigma): D^2v -\mu\cdot\nabla v + f(t,x) =0\\
v(T,x)=g(x)
\end{cases}$

$\sigma(t,x)$ has shape [num_samples,dim,dim]
$dW_t$  has shape [num_samples,dim]

We need to unsqueeze $dW_t$ before batch multiplication.

    
    torch.bmm(sigma(x[:,:,i]),dw[:, :, i]dw[:, :, i].unsqueeze(2))



In [228]:
class sigma(nn.Module): #input [M,D+1]   #output [M,1]
    def __init__(self):
        super(sigma, self).__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(dim+1, 4),
            nn.BatchNorm1d(num_features=4),
            nn.Tanh(),
            nn.Linear(4,dim*dim),
        )
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits#.reshape([dim,dim])

class mu(nn.Module): #input [M,D+1]   #output [M,1]
    def __init__(self):
        super(mu, self).__init__()
        # self.linear_stack = nn.Sequential(
        #     nn.Linear(dim+1, 4),
        #     nn.BatchNorm1d(num_features=4),
        #     nn.Tanh(),
        #     nn.Linear(4,dim),
        # )
    def forward(self, x):
        logits = torch.zeros([x.shape[0],dim])
        return logits
    
class kappa(nn.Module): #input [M,D+1]   #output [M,1]
    def __init__(self):
        super(kappa, self).__init__()
        # self.linear_stack = nn.Sequential(
        #     nn.Linear(dim+1, 4),
        #     nn.BatchNorm1d(num_features=4),
        #     nn.Tanh(),
        #     nn.Linear(4,dim),
        # )
    def forward(self, x):
        logits = torch.zeros([x.shape[0],1])
        return logits
    
class source(nn.Module): #input [M,D+1]   #output [M,1]
    def __init__(self):
        super(source, self).__init__()
        # self.linear_stack = nn.Sequential(
        #     nn.Linear(dim+1, 4),
        #     nn.BatchNorm1d(num_features=4),
        #     nn.Tanh(),
        #     nn.Linear(4,dim),
        # )
    def forward(self, x):
        logits = torch.zeros([x.shape[0],1])
        return logits   
    
class terminal(nn.Module): #input [M,D+1]   #output [M,1]
    def __init__(self):
        super(terminal, self).__init__()
        # self.linear_stack = nn.Sequential(
        #     nn.Linear(dim+1, 4),
        #     nn.BatchNorm1d(num_features=4),
        #     nn.Tanh(),
        #     nn.Linear(4,dim),
        # )
    def forward(self, x):
        logits = -torch.tensor([0.5])*torch.pow(x[:,1:],2)
        return logits 
    
class Y(nn.Module): #input [M,D+1]   #output [M,1]
    def __init__(self):
        super(Y, self).__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(dim+1, 4),
            nn.BatchNorm1d(num_features=4),
            nn.Tanh(),
            nn.Linear(4,1),
        )
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits#.reshape([dim,dim])  
    
class Z(nn.Module): #input [M,D+1]   #output [M,1]
    def __init__(self):
        super(Z, self).__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(dim+1, 4),
            nn.BatchNorm1d(num_features=4),
            nn.Tanh(),
            nn.Linear(4,dim),
        )
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits#.reshape([dim,dim])         

In [229]:
x=torch.rand((8,dim+1))
s = sigma()
m = mu()
k = kappa()
f = source()
g = terminal()
s(x).reshape((8,dim,dim)).shape, m(x).shape, k(x).shape, g(x).shape

z = Z()


In [238]:
class equation(object):
    def __init__(self,sigma,mu,source,kappa,terminal,T,n,a,b,dim):
        num_samples = 2**3
        self.mu = mu
        self.sigma = sigma
        self.kappa = kappa
        self.source = source
        self.x = torch.zeros((num_samples,dim+1,n+1))
        self.Y = Y()
        self.Z = Z()
        dt = torch.tensor([T/n]).to(device)
        dw = torch.randn(size=[num_samples, dim, n]).to(device)* torch.sqrt(dt)
        self.x[:,1:,0]= a+(b-a)*torch.rand(num_samples,dim)
        self.r = torch.ones((num_samples,1,n+1))
        self.c = torch.zeros((num_samples,1,n+1))
        for i in range(n):
            s_tmp = self.sigma(self.x[:,:,i]).reshape((num_samples,dim,dim))
            dw_tmp = dw[:, :, i].unsqueeze(2)
            self.x[:,1:,i+1] = self.x[:,1:,i] + self.mu(self.x[:,:,i])*dt + torch.bmm(s_tmp,dw_tmp).squeeze(2)
            self.x[:,0,i+1] = self.x[:,0,i]+dt
            self.r[:,:,i+1] = self.r[:,:,i]* torch.exp(-self.kappa(self.x[:,:,i])*dt)
            self.c[:,:,i+1] = self.c[:,:,i] + self.source(self.x[:,:,i])*dt
        kc = (self.r*self.c).sum(-1) + g(self.x[:,1:,-1])
        print(kc.mean())
        

        



In [240]:
a = torch.tensor([0.0])
b = torch.tensor([1.0])
heat = equation(s,m,f,k,g,T,num_time_interval,a,b,dim)
heat.Z(heat.x[:,:,1]).shape

tensor(-0.5830, grad_fn=<MeanBackward0>)


torch.Size([8, 2])

In [138]:
dt = torch.tensor([T/num_time_interval]).to(device)
dw = torch.randn(size=[num_sample, dim, num_time_interval]).to(device)* torch.sqrt(dt)
x = torch.zeros((num_sample,dim+1,num_time_interval+1)).to(device)
x[:,1:,0]= a+(b-a)*torch.rand(num_sample,dim)
for i in range(num_time_interval):
    s_tmp = s(x[:,:,i]).reshape((num_sample,dim,dim))
    dw_tmp = dw[:, :, i].unsqueeze(2)
    # print(s_tmp.shape,dw_tmp.shape)
    # print(torch.bmm(s_tmp,dw_tmp).shape)
    # print(m(x[:,:,i]).shape)
    # print(x[:,:,i].shape)
    x[:,1:,i+1] = x[:,1:,i] + m(x[:,:,i])*dt + torch.bmm(s_tmp,dw_tmp).squeeze(2)
    x[:,0,i+1] = x[:,0,i]+dt

In [139]:
x

tensor([[[ 0.0000,  0.5000,  1.0000],
         [ 0.6045,  0.6313,  0.3523],
         [ 0.3737,  0.3689,  0.1290]],

        [[ 0.0000,  0.5000,  1.0000],
         [ 0.0943,  0.4706,  0.8799],
         [ 0.0363,  0.7895,  1.5572]],

        [[ 0.0000,  0.5000,  1.0000],
         [ 0.9576,  0.9594,  0.9705],
         [ 0.3591,  0.3474,  0.2021]],

        [[ 0.0000,  0.5000,  1.0000],
         [ 0.4682,  0.1024, -1.0904],
         [ 0.3310, -0.1241, -1.9073]],

        [[ 0.0000,  0.5000,  1.0000],
         [ 0.5298,  0.7732,  0.5755],
         [ 0.5534,  0.8607,  0.6673]],

        [[ 0.0000,  0.5000,  1.0000],
         [ 0.9525,  0.7066,  0.4719],
         [ 0.7826,  1.2406,  0.8090]],

        [[ 0.0000,  0.5000,  1.0000],
         [ 0.6376,  1.2435,  1.4026],
         [ 0.2813,  0.6142,  1.0592]],

        [[ 0.0000,  0.5000,  1.0000],
         [ 0.1331,  0.5952,  0.4466],
         [ 0.6384,  2.4695,  1.8225]]], grad_fn=<CopySlices>)